# Kubeflow Fairing Introduction

Kubeflow Fairing is a Python package that streamlines the process of `building`, `training`, and `deploying` machine learning (ML) models in a hybrid cloud environment. By using Kubeflow Fairing and adding a few lines of code, you can run your ML training job locally or in the cloud, directly from Python code or a Jupyter notebook. After your training job is complete, you can use Kubeflow Fairing to deploy your trained model as a prediction endpoint.


# How does Kubeflow Fairing work

Kubeflow Fairing 
1. Packages your Jupyter notebook, Python function, or Python file as a Docker image
2. Deploys and runs the training job on Kubeflow or AI Platform. 
3. Deploy your trained model as a prediction endpoint on Kubeflow after your training job is complete.


# Goals of Kubeflow Fairing project

- Easily package ML training jobs: Enable ML practitioners to easily package their ML model training code, and their code’s dependencies, as a Docker image.
- Easily train ML models in a hybrid cloud environment: Provide a high-level API for training ML models to make it easy to run training jobs in the cloud, without needing to understand the underlying infrastructure.
- Streamline the process of deploying a trained model: Make it easy for ML practitioners to deploy trained ML models to a hybrid cloud environment.


> Note: Before fairing workshop, please read `README.md` under `02_01_fairing_introduction`


In [1]:
# check fairing is installed 
!pip show kubeflow-fairing

Name: kubeflow-fairing
Version: 1.0.2
Summary: Kubeflow Fairing Python SDK.
Home-page: https://github.com/kubeflow/fairing
Author: Kubeflow Authors
Author-email: hejinchi@cn.ibm.com
License: Apache License Version 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: urllib3, azure-mgmt-storage, boto3, docker, notebook, oauth2client, retrying, six, cloudpickle, requests, setuptools, google-cloud-storage, grpcio, future, nbconvert, kubeflow-tfjob, python-dateutil, httplib2, numpy, kubeflow-pytorchjob, google-auth, ibm-cos-sdk, kfserving, tornado, azure-storage-file, google-api-python-client, kubernetes, google-cloud-logging
Required-by: 


## Basic Example

If you see any issues, please restart notebook. It's probably because of new installed packages.

Click `Kernel` -> `Restart & Clear Output`

In [15]:
#%%writefile ~/nkodedemo01/nkode/train_model.py
import os
import sys
import tensorflow as tf
import numpy as np

def nkTrain():
    # Genrating random linear data 
    # There will be 50 data points ranging from 0 to 50 
    x = np.linspace(0, 50, 50) 
    y = np.linspace(0, 50, 50) 

    # Adding noise to the random linear data 
    x += np.random.uniform(-4, 4, 50) 
    y += np.random.uniform(-4, 4, 50) 

    n = len(x) # Number of data points 

    X = tf.placeholder("float") 
    Y = tf.placeholder("float")
    W = tf.Variable(np.random.randn(), name = "W") 
    b = tf.Variable(np.random.randn(), name = "b") 
    learning_rate = 0.01
    training_epochs = 1000
    
    # Hypothesis 
    y_pred = tf.add(tf.multiply(X, W), b) 

    # Mean Squared Error Cost Function 
    cost = tf.reduce_sum(tf.pow(y_pred-Y, 2)) / (2 * n)

    # Gradient Descent Optimizer 
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost) 

    # Global Variables Initializer 
    init = tf.global_variables_initializer() 


    sess = tf.Session()
    sess.run(init) 
      
    # Iterating through all the epochs 
    for epoch in range(training_epochs): 
          
        # Feeding each data point into the optimizer using Feed Dictionary 
        for (_x, _y) in zip(x, y): 
            sess.run(optimizer, feed_dict = {X : _x, Y : _y}) 
          
        # Displaying the result after every 50 epochs 
        if (epoch + 1) % 50 == 0: 
            # Calculating the cost a every epoch 
            c = sess.run(cost, feed_dict = {X : x, Y : y}) 
            print("Epoch", (epoch + 1), ": cost =", c, "W =", sess.run(W), "b =", sess.run(b)) 
      
    # Storing necessary values to be used outside the Session 
    training_cost = sess.run(cost, feed_dict ={X: x, Y: y}) 
    weight = sess.run(W) 
    bias = sess.run(b) 

    print('Weight: ', weight, 'Bias: ', bias)

## Local training for development



In [16]:
nkTrain()

Epoch 50 : cost = 8.805625 W = 1.0632132 b = 0.7829763
Epoch 100 : cost = 8.423572 W = 1.0699341 b = 0.4678213
Epoch 150 : cost = 8.102897 W = 1.0760268 b = 0.18212375
Epoch 200 : cost = 7.833296 W = 1.0815501 b = -0.076869205
Epoch 250 : cost = 7.606224 W = 1.086557 b = -0.3116541
Epoch 300 : cost = 7.414626 W = 1.091096 b = -0.5244936
Epoch 350 : cost = 7.252642 W = 1.0952107 b = -0.71743834
Epoch 400 : cost = 7.115422 W = 1.0989408 b = -0.8923491
Epoch 450 : cost = 6.998933 W = 1.1023222 b = -1.050909
Epoch 500 : cost = 6.8998313 W = 1.1053877 b = -1.194651
Epoch 550 : cost = 6.81533 W = 1.1081665 b = -1.3249552
Epoch 600 : cost = 6.7431226 W = 1.1106857 b = -1.4430815
Epoch 650 : cost = 6.6812625 W = 1.1129693 b = -1.5501626
Epoch 700 : cost = 6.6281543 W = 1.1150395 b = -1.6472361
Epoch 750 : cost = 6.5824366 W = 1.1169161 b = -1.7352377
Epoch 800 : cost = 6.5429993 W = 1.1186174 b = -1.8150148
Epoch 850 : cost = 6.50889 W = 1.1201596 b = -1.8873326
Epoch 900 : cost = 6.479325 W =

## Remote training

We will show you how to remotely run training job in kubernetes cluster. You can use `ECR` as your container image registry.

In [23]:
!sh ~/nkodedemo01/nkode/remote_train.sh

~/nkodedemo01/notebooks/Tensor_Flow_Introduction_01/
About to train job setup...
[W 201019 20:49:45 function:49] The FunctionPreProcessor is optimized for using in a notebook or IPython environment. For it to work, the python version should be same for both local python and the python in the docker. Please look at alternatives like BasePreprocessor or FullNotebookPreprocessor.
[W 201019 20:49:45 tasks:62] Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
about to submit job
[I 201019 20:49:45 tasks:66] Building the docker image.
[I 201019 20:49:45 cluster:46] Building image using cluster builder.
[W 201019 20:49:45 base:94] /usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[I 201019 20:49:45 base:107] Creating docker context: /tmp/fairing_context_emlamdjc
[W 201019 20:49:45 base:94] /usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skippi